# Acceso programático a Uniprot

Uniprot también ofrece una REST-API para acceder a sus datos.

Esta interfaz es bastante mas sencilla que la de NCBI.

In [ ]:
library(httr)

In [ ]:
base_url   <- "https://rest.uniprot.org/uniprotkb/search"

Al igual que NCBI, podemos hacer búsquedas por texto, especificando campos lo hacíamos en NCBI, aunque con un formato diferente.

Para escribir nuestra búsqueda tenemos que conocer los campos disponibles:

Estos están en: https://www.uniprot.org/help/query-fields.

El texto de búsqueda tiene que tener una cierta sintaxis: https://www.uniprot.org/help/text-search.

Además tenemos que buscar definir el formato en el que queremos los datos: Tipicamente, en formato tabular y fasta para las secuencias.

Por último que tenemos que definir que datos queremos recuperar: https://www.uniprot.org/help/return_fields

Por ejemplo: queremos recuperar datos de proteínas polymerases de E. coli que

pertenezcan a Swiss-Prot (es decir que estén curadas manualmente).

In [ ]:
query <- 'protein_name:(RNA polymerase) AND (organism_name:"Escherichia coli") AND (reviewed:true)'

respuesta <- GET(
    base_url,
    query=list(query=query, format="tsv", fields="id,accession,protein_name,go_id")
)


In [ ]:
read.table(text=content(respuesta), header=TRUE, sep="\t")

In [ ]:
ecoli_pol <- read.table(text=content(respuesta), header=TRUE, sep="\t")
head(ecoli_pol)

Podemos descargar los secuencias de estas proteinas en formato fasta
cambiando el formato de la respuesta.

[Uniprot reference](https://www.uniprot.org/help/api_queries#list-of-all-formats)

In [ ]:
query <- 'protein_name:(RNA polymerase) AND (organism_name:"Escherichia coli") AND (reviewed:true)'

respuesta <- GET(
    base_url,
    query=list(
        query=query,
        format="fasta",
        fields="id,protein_name"
    )
)


In [ ]:
?httr

In [ ]:
status_code(respuesta)

In [ ]:
# Podemos recuperar las secuencias de los registros de a uno
query <- paste(paste("accession:", ecoli_pol$Entry[1]), collapse=" ")
respuesta <- GET(
    base_url,
    query=list(query=query, format="fasta")
)


In [ ]:
cat(content(respuesta))

Como acceder a las secuencias

In [ ]:
# Para leer formato fasta
# install.packages('seqinr')
# library(seqinr)
sequences <- read.fasta(file=textConnection(content(respuesta, as="text")))

# sequences[[1]]

# summary(sequences[[1]])

# attributes(sequences[[1]])

# attributes(sequences[[1]])

## Servicio IDMapping

Uniprot tiene un servicio que permite mapear identificadores entre diferentes bases de datos.

Es necesario pasarle un lista de identificadores, y decir cuales son las bases

de datos (o tipo de identificadores) de origen y destino.

Lista de los tipos de identificadores que se pueden usar están en:

- https://rest.uniprot.org/configure/idmapping/fields

In [ ]:
library(httr2)

In [ ]:
req <- request(url)

req %>% req_body_multipart(
  ids="P21802",
  from="UniProtKB_AC-ID",
  to="UniRef90"
) %>%
  req_perform()

In [ ]:
url <- "https://rest.uniprot.org/idmapping/run"

# Define the parameters
params <- list(
  ids="P21802",
  from="UniProtKB_AC-ID",
  to="UniRef90"
)

# Perform the POST request
response <- POST(
  url ,
  body = params,
  encode="multipart"
)


In [ ]:
response

In [ ]:
retrieve_url <- 'https://rest.uniprot.org/idmapping/run/'
POST(
    url=retrieve_url,
    body=list(
        ids='P21802,P12345',
        from='UniProtKB_AC-ID',
        to='UniRef90'
    ),
    encode="json"
)


In [ ]:
content(respuesta)

In [ ]:
mapping <- head(read.table(text=content(respuesta), header=TRUE))

In [ ]:
mapping

## Paquetes de R para conectarse a Uniprot

Hay dos paquetes de R que permiten conectarse a Uniprot:

- Uniprot.ws: https://www.bioconductor.org/packages/release/bioc/html/UniProt.ws.html
- UniprotR: https://cran.r-project.org/web/packages/UniprotR/index.html

No tienen una funcionalidad muy completa, por ejemplo carecen de la capacidad de hacer búsquedas de texto.

Aunque permiten descargar datos si se tienen los identificadores de los registros.